In [22]:
from pydrake.all import ModelVisualizer, Simulator, StartMeshcat
from manipulation import ConfigureParser, running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation

from typing import Callable

import numpy as np
from pydrake.all import (
    BasicVector,
    Box,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SnoptSolver,
    PiecewisePolynomial,
    ModelInstanceIndex,
    TrajectorySource,
    AddFrameTriadIllustration,
    Trajectory,
    PiecewisePose,
    ge,
    le,
    Solve,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_differential_ik import TestDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import RenderDiagram
from pydrake.multibody.tree import JacobianWrtVariable
import numpy as np

In [23]:
def design_grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    fill in our code below
    """
    R_OG = RotationMatrix.MakeXRotation(-np.pi/2)

    p_WO = X_WO.translation()
    R_WO = X_WO.rotation()
    p_WG = p_WO + np.array([0.0, 0.0, 0.05])
    p_OG = R_WO.inverse() @ (p_WG - p_WO)

    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @X_OG
    return X_OG, X_WG

In [24]:
def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    """
    fill in our code below
    """
    X_GGapproach = RigidTransform(p=np.array([0.0, -0.1, 0.0]))
    X_WGApproach = X_WG @ X_GGapproach
    return X_WGApproach

In [25]:
def make_trajectory(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    robot_velocity_trajectory = None
    traj_wsg_command = None
    # TODO: define a PiecewisePose out of the X_Gs
    pose_traj = PiecewisePose.MakeLinear(sample_times, X_Gs)


    # TODO: set robot_velocity_trajectory to the derivative of the pose trajectory you just defined
    robot_velocity_trajectory: Trajectory = pose_traj.MakeDerivative()



    # TODO: set traj_wsg_command to a PiecewisePolynomial that commands the fingers
    knots = np.asarray(finger_values, dtype=float).reshape(1, -1)  # (1, L)
    traj_wsg_command = PiecewisePolynomial.ZeroOrderHold(sample_times, knots)



    return robot_velocity_trajectory, traj_wsg_command

In [26]:
scenario_data = """
        directives:
        - add_model:
            name: iiwa
            file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
            default_joint_positions:
                iiwa_joint_1: [-1.57]
                iiwa_joint_2: [0.1]
                iiwa_joint_3: [0]
                iiwa_joint_4: [-1.2]
                iiwa_joint_5: [0]
                iiwa_joint_6: [1.6]
                iiwa_joint_7: [0]

        - add_model:
            name: wsg
            file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
            
        - add_weld:
            parent: world
            child:  iiwa::iiwa_link_0

        - add_weld:
            parent: iiwa::iiwa_link_7
            child: wsg::body
            X_PC:
                translation: [0, 0, 0.09]
                rotation: !Rpy { deg: [90, 0, 90]}

        - add_model:
            name: hoop_model
            file: file:///workspaces/6.4210-final-project/sdfs/basketball_hoop.sdf

        - add_weld:
            parent: world
            child: hoop_model::base_link_hoop  
            X_PC:
                translation: [0, -5, 3.048]
                rotation: !Rpy { deg: [0, 0, 90] }

        - add_model:
            name: ball
            file: package://drake_models/manipulation_station/sphere.sdf
            default_free_body_pose:
                base_link:
                    translation: [0.5, 0, 0]
                    rotation: !Rpy { deg: [0, 0, 0] }

        - add_model:
            name: table
            file: file:///workspaces/6.4210-final-project/sdfs/table.sdf
        - add_weld:
            parent: world
            child: table::table_link
            X_PC:
                translation: [1, 0.0, -0.05]
                rotation: !Rpy { deg: [0, 0, -90] }

        model_drivers:
            iiwa: !IiwaDriver
                hand_model_name: wsg
            wsg: !SchunkWsgDriver {}
    """

scenario = LoadScenario(data=scenario_data)

In [27]:
# Helper function to express mesh poses in terms of COM rather than geometric center


def get_initial_pose(
    plant: MultibodyPlant,
    body_name: str,
    model_instance: ModelInstanceIndex,
    plant_context: Context,
) -> RigidTransform:
    body = plant.GetBodyByName(body_name, model_instance)
    X_WS = plant.EvalBodyPoseInWorld(plant_context, body)
    X_SO = RigidTransform(body.default_spatial_inertia().get_com())
    return X_WS @ X_SO

In [28]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()
        
        # Allows us to only use certain joints at different phases
        self.mask_align = np.array([0, 0, 0, 1, 1, 1, 1]) 
        self.mask_throw = np.array([1, 1, 1, 0, 0, 0, 0])

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        fill in our code below.
        """
        # evaluate the V_G_port and q_port on the current context to get those values.

        V_WG_des = np.asarray(self.V_G_port.Eval(context)).reshape(6)   # [w; v] expressed in W
        q = np.asarray(self.q_port.Eval(context)).reshape(7)            # iiwa positions

        # update the positions of the internal _plant_context according to `q`.
        # HINT: you can write to a plant context by calling `self._plant.SetPositions`
        self._plant.SetPositions(self._plant_context, self._iiwa, q)

        # Compute the gripper jacobian
        # HINT: the jacobian is 6 x N, with N being the number of DOFs.
        # We only want the 6 x 7 submatrix corresponding to the IIWA
        #    V_WG_W = J_WG * v  (J is 6 x nv).
        J_WG_full = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kV,
            self._G,                   # frame B (gripper)
            np.zeros(3),               # p_BG (point: origin of G)
            self._W,                   # frame A (world)
            self._W                    # expressed in W
        )
        J = J_WG_full[:, self.iiwa_start : self.iiwa_end + 1]  # 6x7


        # compute `v` by mapping the gripper velocity (from the V_G_port) to the joint space
        lam2 = 1e-4
        #    v = Jᵀ (J Jᵀ + λ² I)⁻¹ V
        v = J.T @ np.linalg.solve(J @ J.T + lam2 * np.eye(6), V_WG_des)
        
        # t = context.get_time()
        # if t < 10.0:
        mask = np.array([1, 1, 1, 1, 1, 1, 1])
        # else:
        #     mask = self.mask_align if t < 12.0 else self.mask_throw

        v = v * mask

        output.SetFromVector(v)

In [29]:
g = [0, 0, -9.81]

def calculate_ball_release_pose(X_H, X_B, entry_angle, s_hit, T_bounds):
    prog = MathematicalProgram()
    v_rel = prog.NewContinuousVariables(3, "v_rel")
    T = prog.NewContinuousVariables(1, "T")[0]
    
    # Get entry direction for a given angle
    p_hit = X_H.translation()
    R_WH = X_H.rotation()
    z_H = R_WH.col(2)
    
    down = -z_H / np.linalg.norm(z_H)
    
    world_up = np.array([0., 0., 1.])
    horizontal = np.cross(down, world_up)
    
    v_dir = np.cos(entry_angle) * down + np.sin(entry_angle) * horizontal
    v_dir = v_dir / np.linalg.norm(v_dir)
    
    v_hit = s_hit * v_dir
    
    # projectile constraints
    T_min, T_max = T_bounds
    prog.AddBoundingBoxConstraint(T_min, T_max, T)
    
    p0 = X_B.translation()
    
    for i in range(3):
        prog.AddConstraint(v_rel[i] + g[i] * T == v_hit[i])
        prog.AddConstraint(p0[i] + v_rel[i] * T + 0.5 * g[i] * T**2 == p_hit[i])
        
    # Solve for v_rel
    prog.AddQuadraticCost(v_rel @ v_rel)
    result = Solve(prog)
    
    p_rel_val = p0.copy()
    v_rel_val = result.GetSolution(v_rel)
    T_val = result.GetSolution(T)

    return p_rel_val, v_rel_val, T_val


def calculate_hoop_alignment_pose(X_H, X_B) -> RigidTransform:
    """
    Returns X_WG_align such that the gripper's z-axis points toward the hoop
    (in the horizontal plane), and y-axis is aligned with world up.
    """
    p_hit = X_H.translation()
    p_B = X_B.translation()

    throw_dir = p_hit - p_B
    throw_dir[2] = 0.0
    throw_norm = np.linalg.norm(throw_dir)
    z_G = -throw_dir / throw_norm

    y_G = np.array([0., 0., 1.])

    x_G = np.cross(y_G, z_G)
    x_norm = np.linalg.norm(x_G)
    x_G = x_G / x_norm

    z_G = np.cross(x_G, y_G)
    z_G = z_G / np.linalg.norm(z_G)

    R_WG_align = RotationMatrix(np.column_stack((x_G, y_G, z_G)))
    p_WG_align = p_B.copy() 

    return RigidTransform(R_WG_align, p_WG_align)


In [30]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
# Define the builder we will use to specify the full diagram.
# Add the hardware station to the diagram
builder = DiagramBuilder()
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

# get initial poses of gripper and objects
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
model_instance_ball = plant.GetModelInstanceByName("ball")
X_WOball_initial = get_initial_pose(
    plant, "base_link", model_instance_ball, temp_plant_context
)


# Build trajectory keyframes
X_OG, X_WG_pick = design_grasp_pose(X_WOball_initial)
X_WG_prepick = approach_pose(X_WG_pick)

X_H = get_initial_pose(
    plant, "base_link_hoop", plant.GetModelInstanceByName("hoop_model"), temp_plant_context
)

X_GO = X_OG.inverse()
X_WO_hold = X_WGinitial @ X_GO
X_WG_align = calculate_hoop_alignment_pose(X_H, X_WO_hold)

p_rel, v_rel, T = calculate_ball_release_pose(X_H, X_WO_hold, 45, 5.0, [0.5, 4.0])
# R_WG_release = X_WG_align.rotation()
# p_WG_release = p_rel
# X_WG_release = RigidTransform(R_WG_release, p_WG_release)
# ball position at release = p_rel
p_ball_release = p_rel

# ball center relative to gripper, expressed in the aligned frame
p_GO = X_GO.translation()
R_WG_align = X_WG_align.rotation()

# correct gripper origin at release
p_WG_release = p_ball_release - R_WG_align @ p_GO

# orientation stays aligned
X_WG_release = RigidTransform(R_WG_align, p_WG_release)

throw_dir_W = -(R_WG_align.col(2))   # -z_G direction
throw_dir_W /= np.linalg.norm(throw_dir_W)
p_WG_prethrow = p_WG_release - 0.3 * throw_dir_W
X_WG_prethrow = RigidTransform(R_WG_align, p_WG_prethrow)




# constants for finger distances when the gripper is opened or closed
opened = 0.107
closed = 0.0

# list of keyframes, formatted as (gripper poses, finger states)
# for each object the robot starts in its default pose with its gripper open
# then it goes to the prepick pose, the pick pose, closes the gripper, and then goes
# to the place pose
keyframes = [
    (X_WGinitial,  opened),  # start at home, gripper open
    (X_WG_prepick, opened),  # move above/behind the ball
    (X_WG_pick,    opened),  # descend onto the ball
    (X_WG_pick,    closed),  # close on the ball
    (X_WG_prepick, closed),  # lift back up with ball grasped
    (X_WGinitial,  closed),  
    (X_WG_align,  closed),  
    (X_WG_prethrow,  closed),  
    (X_WG_release,   closed),  
    (X_WG_release,  opened),  
]

# unpack the keyframes and use them to build `Trajectory` objects
# note: we specify each keyframe as occuring 2 seconds after the last.
gripper_poses = [keyframe[0] for keyframe in keyframes]
finger_states = np.asarray([keyframe[1] for keyframe in keyframes]).reshape(1, -1)
sample_times = [2 * i for i in range(len(gripper_poses))]
traj_V_G, traj_wsg_command = make_trajectory(gripper_poses, finger_states, sample_times)

# V_G_source defines a trajectory over gripper velocities. Add it to the system.
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
# Add the DiffIK controller we just defined to the system
controller = builder.AddSystem(PseudoInverseController(plant))
# The HardwareStation expects robot commands in terms of joint angles.
# We define the `integrator` system to map from joint_velocities to joint_angles.
integrator = builder.AddSystem(Integrator(7))
# wsg_source defines a trajectory of finger positions. Add it to the system.
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# TODO: connect the joint velocity source to the pseudoinverse controller
builder.Connect(
    V_G_source.get_output_port(),
    controller.GetInputPort("V_WG"),
)

# TODO: connect the controller to integrator to get joint angle commands
builder.Connect(
    controller.GetOutputPort("iiwa.velocity"),
    integrator.get_input_port(),
)

# TODO: connect the joint angles computed by the integrateor to the iiwa.position port on the manipulation station
builder.Connect(
    integrator.get_output_port(),
    station.GetInputPort("iiwa.position"),
)

# TODO: connect the "iiwa.position_measured" port on the station back to the relevant input port on the controller
builder.Connect(
    station.GetOutputPort("iiwa.position_measured"),
    controller.GetInputPort("iiwa.position"),
)

zero_torque = builder.AddSystem(ConstantVectorSource(np.zeros(7)))
builder.Connect(
    zero_torque.get_output_port(),
    station.GetInputPort("iiwa.torque"),
)

# TODO: connect the wsg_source to the "wsg.position" input port of the station
builder.Connect(
    wsg_source.get_output_port(),
    station.GetInputPort("wsg.position"),
)


# visualize axes (useful for debugging)
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName("base_link", model_instance_ball),
    length=0.1,
)

AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
)

diagram = builder.Build()

In [32]:
# Define the simulator.
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant.GetMyContextFromRoot(context),
        plant.GetModelInstanceByName("iiwa"),
    ),
)
diagram.ForcedPublish(context)
print(f"sanity check, simulation will run for {traj_V_G.end_time()} seconds")

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

sanity check, simulation will run for 6.6 seconds
